<h2>Flask</h2>
<h4>Dependencies</h4>

In [14]:
!pip install fastai
!pip install flask

<h3>Imports</h3>
The imports for the flask website.
We used os to make directories and re to do some regex on the uploaded files.

There was also a dependency issue with load_learner from fastai.vision.all,
tt used PosixPath but that isn't used in windows so we fixed it by replacing PosixPath with the WindowsPath.

In [17]:
from flask import Flask, render_template, request, redirect, url_for
from werkzeug.utils import secure_filename
import os
import re


import random as rnd
from fastai import *
from fastai.vision.all import *

# fixes a dependancy on PosixPath by load_learner
import pathlib
temp = pathlib.PosixPath
pathlib.PosixPath = pathlib.WindowsPath

<h3>Global variables</h3>

These contain fixed data that is used by the website.
The image that is uploaded on the website to be tested by the model is saved in the uploadfolder.

In [18]:
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = './static/UploadFolder'

DATAPATH = "./static/data/"
# These are the categories used in the model and the data
CATEGORIES = os.listdir(DATAPATH)
ALLOWED_IMGTYPES = ['png','jpeg','gif','jfif','jpg']
# uses top 20 images from the scrapers
N_MAX = 20

# learn is the model that is loaded from the pkl file. this file contains a model that predicts 14 different archtectal styles
learn = load_learner('exportBig.pkl')

<h3>Format functions</h3>

These functions are used to format the data that has to be represented in the webpage.
Example is a 'card' containing the right category, the image and the predicted category.

Sample is the formatting of the uploaded image to be tested.

In [19]:
def format_example(imgpath,cat,prediction):
   return {"image":"/data/"+imgpath,
                           "category":cat,
                           "prediction":prediction[0],
                           "correct_class":"example_"+str(cat==prediction[0])}

def format_sample(imgpath,prediction):
   return {"image":"/UploadFolder/"+imgpath,
            "prediction":prediction[0],
            "sample_class":"sample_visable"}

<h3>Examples function</h3>

This function gets a random image from every category and returns a list of all the formatted examples.

In [20]:
def get_examples():
   examples = []
   for cat in CATEGORIES:
      n = rnd.choice(range(N_MAX))
      imgpath = cat+"/"+cat+str(n)+".png"
      prediction = learn.predict(DATAPATH+imgpath)

      examples.append(format_example(imgpath,cat,prediction))
   
   return examples

<h3>Flask routed functions</h3>

These functions are routed by Flask and return the same template (home).
The difference is that the initial routing is used when no image is uploaded and the second function is used when an image is uploaded.


In [21]:
@app.route('/')
def home():
   examples = get_examples()
   return render_template('home.html',examples=examples,sample={"sample_class":"sample_hidden"})

@app.route("/testImage_post", methods=['post'])
def testImage_post():

   # clears the upload folder, to prevent to much space being used
   for img in os.listdir(app.config['UPLOAD_FOLDER']):
      os.remove(os.path.join(app.config['UPLOAD_FOLDER'],img))

   # requests file from the posted files, this is suposed to be an image
   image = request.files['file']
   imgpath = secure_filename(image.filename)

   #regex function to see if uploaded file was an image, else return to home function
   if re.sub('.*\\.','',imgpath) not in ALLOWED_IMGTYPES:
      return redirect(url_for('home'))

   image.save(os.path.join(app.config['UPLOAD_FOLDER'], imgpath))

   #test the posted image with the model
   prediction = learn.predict(os.path.join(app.config['UPLOAD_FOLDER'], imgpath))

   sample = format_sample(imgpath,prediction)
   examples = get_examples()
   return render_template('home.html',examples=examples,sample=sample)

<h3>Run the server</h3>

In [22]:

if __name__ == '__main__':
   app.run()

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


127.0.0.1 - - [12/Dec/2021 19:46:21] "POST /testImage_post HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static//data/ancient%20egyptian/ancient%20egyptian7.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static//data/tudor/tudor5.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static//data/gothic/gothic1.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static//data/federal/federal11.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static/UploadFolder/OnzeLieveVrouwetoren_Amersfoort-683x1024.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static/data/roman/roman7.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static/data/indoislamic/indoislamic10.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static/data/tudor/tudor5.png HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:46:21] "GET /static/data/

127.0.0.1 - - [12/Dec/2021 19:46:38] "POST /testImage_post HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:46:38] "GET /static//data/ancient%20egyptian/ancient%20egyptian0.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:38] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 19:46:39] "GET /static//data/roman/roman8.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:39] "GET /static//data/tudor/tudor15.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:39] "GET /static//data/indoislamic/indoislamic4.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:39] "GET /static//data/gothic/gothic18.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:39] "GET /static/UploadFolder/image.jpeg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:46:39] "GET /static/data/federal/federal2.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 19:46:39] "GET /static/data/roman/roman8.png HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:46:39] "GET /static/data/tudor/tudor15.png HTTP/1.1" 200 -

127.0.0.1 - - [12/Dec/2021 19:46:54] "POST /testImage_post HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static//data/ancient%20egyptian/ancient%20egyptian14.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static/css/style.css HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static//data/gothic/gothic10.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static//data/tudor/tudor0.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static//data/indoislamic/indoislamic2.png HTTP/1.1" 308 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static/data/federal/federal7.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static/UploadFolder/OnzeLieveVrouwetoren_Amersfoort-683x1024.jpg HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static/data/roman/roman5.png HTTP/1.1" 304 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static/data/gothic/gothic10.png HTTP/1.1" 200 -
127.0.0.1 - - [12/Dec/2021 19:46:54] "GET /static/da